# Lecture 4 - Command Line Arguments

## 📕 Today's Agenda
---
 * [Simple argument parsing](#Simple-argument-parsing)
 * [Parsing with argparse](#Parsing-with-argparse)
 * [Script version and description](#Script-version-and-description)
 * [Mandatory and optional arguments](#Mandatory-and-optional-arguments)
 * Actions and default values
 * Argument validation

## 🧪 Theory
---

### Simple argument parsing

One of the best feature of CLI tools is the ability of injecting data into a program. This
is a key point in automation world.
Parsing arguments can be messy when the number of arguments is big, but for one or two positional arguments
is enough to use `sys` module.

In [1]:
import sys

print(sys.argv)
print('First arg:', sys.argv[1])

for arg in sys.argv:
    print(arg)

['c:\\users\\mdinu\\appdata\\local\\programs\\python\\python39\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\mdinu\\AppData\\Roaming\\jupyter\\runtime\\kernel-537ca260-35e5-49cd-8ee9-f34d1fff71ed.json']
First arg: -f
c:\users\mdinu\appdata\local\programs\python\python39\lib\site-packages\ipykernel_launcher.py
-f
C:\Users\mdinu\AppData\Roaming\jupyter\runtime\kernel-537ca260-35e5-49cd-8ee9-f34d1fff71ed.json


### Parsing with argparse
The simplest and fastest path to a good argument parsing is usage of `argparse` module. It
provides a simple interface for argument handling if configured well.
It comes with out of the box support for common arguments like -h (--help) and -v (--version).

Let's jump in and create a simple arg parser.

NOTE: `parse_args` should be called without arguments when getting arguments from CLI. In this notebook
i can't pass arguments to the CLI so I will pass them into `parse_args` function call.


In [2]:
import argparse
argparser = argparse.ArgumentParser()
argparser.add_argument('name')
argparser.add_argument('surname')
args = argparser.parse_args(['mihai', 'dinu'])
print(args)
print(args.name)
print(args.surname)

Namespace(name='mihai', surname='dinu')
mihai
dinu


Code explained:
Line 1: create a new ArgumentParser object, this will hold info about available arguments.
Line 2: add first positional argument to the parser.
Line 3: add second positional argument to the parser.
Line 4: parse CLI args and store data into parser.
Line 5-7: Use data passed via arguments.

### Script version and description
As I already discussed about built-in ability to handle version and help, let's configure them.
Help is constructed using available arguments in parser. Use *help* kwarg to specify a help message for an argument.

In [6]:
argparser2 = argparse.ArgumentParser(description='Notebooks description.')
argparser2.add_argument('name', help='User name')
argparser2.parse_args(['--help'])

usage: ipykernel_launcher.py [-h] name

Notebooks description.

positional arguments:
  name        User name

optional arguments:
  -h, --help  show this help message and exit


SystemExit: 0

c:\users\mdinu\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


For version handling a there in not a already defined argument, we have to define it, instead exists an action.

In [4]:
argparser2.add_argument('-v', action='version', version='1.0')
argparser2.parse_args(['-v'])

1.0


SystemExit: 0

c:\users\mdinu\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Mandatory and optional arguments
Each optional argument can trigger one action or can pass a value with it. For example --verbose can set
a variable to True but if you provide a number after it (--verbose 2), the number can be used to set verbosity level.
To specify how much sub-arguments an argument can take `nargs` kwarg must be provided. Along `nargs` argument, `meta` kwarg
is needed, but not mandatory, to describe what is expected for that CLI arg.

In [10]:
argparser3 = argparse.ArgumentParser()
argparser3.add_argument(
    '-n',
    nargs=1,
    metavar='NUMBERS',
    help="Generate n numbers."
)
argparser3.add_argument(
    '-r',
    '--read',
    help='Read files.'
)
args = argparser3.parse_args(['-h'])

usage: ipykernel_launcher.py [-h] [-n NUMBERS] [-r READ]

optional arguments:
  -h, --help            show this help message and exit
  -n NUMBERS            Generate n numbers.
  -r READ, --read READ  Read files.


SystemExit: 0

c:\users\mdinu\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 👩‍💻 Practice
---
1. Write a script that takes two arguments. First argument is a number and the second one is a path to an output file.
The script will compute Fibonacci numbers up to first argument and will write them on the output file
specified by the second argument, one number per line. Argument parsing will be done using *sys* module. Make sure you check
for the number of arguments provided by user and make sure that they are valid. For Fibonacci calculus make a generator function.

2. Modify the code written for point 1 in order to use `argparse`.

## 🏠 Homework
---